In [2]:
## Import libraries & files

import pandas as pd
from ipyaggrid import Grid
from mlxtend.frequent_patterns import apriori, association_rules

excel_path = r"C:/Users/LuanHy/Desktop/FocusSpace/1. Interview/Test/UrBox_Test/Test_Answers/Test_Data.xlsx"


In [3]:
## Function

def summarize_user(g):
    g = g.sort_values(["Redeemed_time_rank", "Min_redeemed_time"])

    rows_first = g.loc[g["Redeemed_time_rank"] == 1]
    if not rows_first.empty:
        first_brand_id = rows_first["brand_id"].iloc[0]
        first_brand_date = rows_first["Min_redeemed_time"].iloc[0]
    else:
        first_brand_id = None
        first_brand_date = None

    rows_second = g.loc[g["Redeemed_time_rank"] == 2]
    if not rows_second.empty:
        second_brand_id = rows_second["brand_id"].iloc[0]
        second_brand_date = rows_second["Min_redeemed_time"].iloc[0]
    else:
        second_brand_id = None
        second_brand_date = None

    last_row = g.iloc[-1]
    last_brand_id = last_row["brand_id"]
    last_redeemed_date = last_row["Max_redeemed_time"]
    number_of_brands = len(g)

    return pd.Series(
        {
            "first_brand_id": first_brand_id,
            "second_brand_id": second_brand_id,
            "first_brand_redeemed_date": first_brand_date,
            "second_brand_redeemed_date": second_brand_date,
            "last_brand_id": last_brand_id,
            "last_redeemed_date": last_redeemed_date,
            "number_of_brands": number_of_brands,
        }
    )



In [4]:
## Transaction_table

Transactions_Data = pd.read_excel(excel_path,sheet_name="Data for Part 1")

In [5]:
## User_table

User_Data = (
    Transactions_Data
        .groupby(["user_id","brand_id"],as_index=False)
        .agg(
            Min_redeemed_time = ("voucher_redeemed_at","min"),
            Max_redeemed_time = ("voucher_redeemed_at","max"),
        )
        .sort_values(["user_id", "Min_redeemed_time"])
        .assign(
            Redeemed_time_rank = lambda d:d
                .groupby("user_id")["Min_redeemed_time"]
                .rank(method="dense",ascending=True)
        )
)

User_Data = (
    User_Data
      .groupby("user_id")
      .apply(summarize_user, include_groups=False)
      .reset_index()
)

User_Data = (
    User_Data
        .assign(
            user_age = (
                pd.to_datetime(User_Data["last_redeemed_date"])
                - pd.to_datetime(User_Data["first_brand_redeemed_date"])
            ).dt.days,
        age_group = lambda d: pd.cut(
                (
                    pd.to_datetime(d["last_redeemed_date"])
                    - pd.to_datetime(d["first_brand_redeemed_date"])
                ).dt.days,
                bins=[-1, 7, 30, 90,9999],
                labels=["New User", "Early User", "Growth User","Loyal User"]
            )
        )
    ) 
User_Data = User_Data.sort_values("user_age",ascending=True)
User_Data


,user_id,first_brand_id,second_brand_id,first_brand_redeemed_date,second_brand_redeemed_date,last_brand_id,last_redeemed_date,number_of_brands,user_age,age_group
34,12315078,27,654.0,2025-07-06 04:05:49,2025-07-06 04:22:38,1511,2025-07-06 04:43:45,3,0,New User
40,20474701,82,NaN,2025-05-04 21:27:18,None,82,2025-05-04 21:36:22,1,0,New User
983,1014885635,274,NaN,2025-07-29 00:17:36,None,274,2025-07-29 00:17:36,1,0,New User
545,1006557974,910,NaN,2025-05-23 19:33:46,None,910,2025-05-23 19:36:49,1,0,New User
980,1014846361,392,NaN,2025-05-09 19:27:07,None,392,2025-05-09 19:27:07,1,0,New User
...,...,...,...,...,...,...,...,...,...,...
727,1008996575,397,552.0,2025-01-01 18:44:54,2025-01-09 21:21:05,673,2025-07-26 21:54:13,5,206,Loyal User
152,1000932433,552,7.0,2025-01-02 01:36:50,2025-01-21 04:31:49,992,2025-07-27 22:47:08,3,206,Loyal User
719,1008887807,552,1529.0,2025-01-04 18:01:21,2025-01-06 02:56:01,992,2025-07-31 02:06:58,5,207,Loyal User
216,1001699208,552,82.0,2025-01-02 15:28:32,2025-01-05 22:23:26,992,2025-07-29 15:49:55,4,208,Loyal User


In [6]:
# User_and_Transaction_Numbers_Pivot

User_and_Transaction_Numbers = (
    Transactions_Data
        .merge(
            User_Data[["user_id", "age_group"]],
            on="user_id",
            how="left"
        )
        .groupby("age_group",observed=False)
        .agg(
            user_count = ("user_id", "nunique"),
            transaction_count = ("transaction_id", "count"),
        )
        .assign(
            avg_transactions_per_user = lambda d: (d["transaction_count"] / d["user_count"]).round(1)
        )
        .reset_index()
)

User_and_Transaction_Numbers


,age_group,user_count,transaction_count,avg_transactions_per_user
0,New User,481,898,1.9
1,Early User,80,416,5.2
2,Growth User,185,1303,7.0
3,Loyal User,254,3035,11.9


In [7]:
## Join 2 tables

Merge_Table = (
    Transactions_Data
        .merge(
            User_Data[["user_id", "age_group"]],
            on="user_id",
            how="left"
        )
    )

Merge_Table


,transaction_id,user_id,brand_id,voucher_redeemed_at,age_group
0,136842105,108465,397,2025-01-19 18:44:23,New User
1,136965393,108465,397,2025-01-22 04:15:14,New User
2,136965394,108465,397,2025-01-22 04:15:14,New User
3,137039982,108465,397,2025-01-23 20:05:49,New User
4,137039983,108465,397,2025-01-23 20:05:49,New User
...,...,...,...,...,...
5647,144196927,1016178083,1511,2025-07-03 16:26:19,Early User
5648,145127671,1016194503,82,2025-07-24 16:35:23,New User
5649,144022380,1016336368,1511,2025-06-30 19:06:36,Early User
5650,145129746,1016336368,1511,2025-07-24 17:35:47,Early User


In [8]:
## Brand count by age_group

Data_table = (
    Merge_Table
        .groupby(["age_group"],observed=False)
        .agg(
            brand_count=("brand_id", "nunique"),
        )
        .reset_index()
)
Data_table


,age_group,brand_count
0,New User,48
1,Early User,43
2,Growth User,47
3,Loyal User,59


In [9]:
## Top5_By_AgeBrand_Data

Data_table = (
    Merge_Table
        .groupby(["age_group", "brand_id"],observed=False)
        .agg(
            transaction_count=("transaction_id", "count"),
        )
        .reset_index()
)

Top5_By_AgeBrand = (
    Data_table
        .sort_values(["age_group", "transaction_count"], ascending=[True, False])
        .groupby("age_group",observed=False)
        .head(5)   # lấy 5 dòng đầu của mỗi age_group sau khi sort
        .reset_index(drop=True)
)

Top5_By_AgeBrand



,age_group,brand_id,transaction_count
0,New User,1511,276
1,New User,395,99
2,New User,82,93
3,New User,882,52
4,New User,910,38
5,Early User,82,71
6,Early User,1511,48
7,Early User,882,38
8,Early User,395,36
9,Early User,552,34


In [10]:
## Apply Apriori methods for Basket Analysis

Transactions_Data = Transactions_Data.dropna(subset=['user_id', 'brand_id'])

Transactions_Data_unique = Transactions_Data.drop_duplicates(subset=['user_id', 'brand_id']).copy()

## User with list brand check
print("------ USER WITH LIST BRAND CHECK ------")
print(Transactions_Data_unique.groupby('user_id')['brand_id'].apply(list).head(10))

## Create Maxtrix for Apriori
Transactions_Data_unique['value'] = 1
basket = Transactions_Data_unique.pivot_table(
    index='user_id',
    columns='brand_id',
    values='value',
    fill_value=0
)
basket = basket.astype(bool)
print("\n\n------ MATRIX CHECK ------")
print(basket.head())

## Run Apriori and setup min_support = 0.02 to find itemset >= 2% users
freq_items = apriori(
    basket,
    min_support=0.02,
    use_colnames=True
)
freq_items = freq_items.sort_values(by='support', ascending=False)
print("\n\n=== FREQUENT ITEMSETS CHECK===")
print(freq_items.head(20))

## 7. Create ASSOCIATION RULES (SUPPORT, CONFIDENCE, LIFT)
rules = association_rules(
    freq_items,
    metric="lift",
    min_threshold=1.0
)

rules = rules.sort_values(by='lift', ascending=False)

print("\n\n------ ASSOCIATION RULES CHECK ------")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(30))

## STRONG RULES
strong_rules = rules[
    (rules['support'] >= 0.02) &
    (rules['confidence'] >= 0.3) &
    (rules['lift'] >= 1.2)
].copy()

strong_rules = strong_rules.sort_values(by='lift', ascending=False)

print("\n\n------ STRONG RULES CHECK ------")
print(strong_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))


------ USER WITH LIST BRAND CHECK ------
user_id
108465           [397]
135042           [890]
218587            [23]
410610            [82]
427212     [992, 1511]
539864          [1511]
735746           [396]
890259           [593]
1385395      [34, 315]
1537715          [395]
Name: brand_id, dtype: object


------ MATRIX CHECK ------
brand_id   0      1      7      23     24     27     30     34     35    \
user_id                                                                   
108465    False  False  False  False  False  False  False  False  False   
135042    False  False  False  False  False  False  False  False  False   
218587    False  False  False   True  False  False  False  False  False   
410610    False  False  False  False  False  False  False  False  False   
427212    False  False  False  False  False  False  False  False  False   

brand_id   36    ...   1054   1068   1070   1110   1170   1256   1292   1511  \
user_id          ...                                    